In [55]:
import os
import time
import cv2
from mss import mss
import numpy as np
import keyboard

In [41]:
dinogame_coords = {
    "top": 360, 
    "left": 166, 
    "width": 500, 
    "height": 80
}
sct = mss()

In [44]:
img = np.array(sct.grab(dinogame_coords))


In [45]:
cv2.imwrite('test.jpg', img)

True

In [31]:
cv2.destroyAllWindows()

In [11]:
import datetime
now = datetime.datetime.now()
str(now.year)+str(now.day)+str(now.month)

'20202610'

In [13]:
# test reading in the image and see what it looks like..
#  
img_num = 6
img = cv2.imread(f'../images/frame_{img_num}.jpg', cv2.IMREAD_GRAYSCALE)
img.shape

(106, 500)

In [14]:
# original image
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
img_resize = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)
img_resize.shape

(53, 250)

In [16]:
# original scaled down 50%
cv2.imshow('img', img_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
img_reshape = img_resize[:, :, np.newaxis]
img_reshape.shape

(53, 250, 1)

In [22]:
# reshaped image? still works on cv2
cv2.imshow('img', img_reshape)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
img_reshape2 = img_resize.flatten()
img_reshape2.shape

(13250,)

In [24]:
img_fix = img_reshape2.reshape(53, 250)
img_fix.shape

(53, 250)

In [25]:
# this puts the image back. 
cv2.imshow('img', img_fix)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
# next... store the flattened images in a databasepi
import pandas as pd
import fastparquet as fp

In [33]:
# put this together
imagelist = []

for img_num in np.arange(0,10):
    img = cv2.imread(f'../images/frame_{img_num}.jpg', cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, dsize=None, fx=0.5, fy=0.5)
    img = img.flatten()
    imagelist.append(img)
    

imagelist = np.asarray(imagelist)

In [35]:
imagelist

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [34]:
cv2.imshow('img', imagelist[3].reshape(53, 250))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
df = pd.DataFrame(data=imagelist)
df.columns = df.columns.astype(str)

In [43]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13240,13241,13242,13243,13244,13245,13246,13247,13248,13249
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# need to appendatetime to file name
# need to attach a column to dataframe

fp.write("../images/testgameplay.parq", df, compression='GZIP')

## Learning --> It's way more efficient to store as jpegs!!

In [50]:
# better to just make a folder for each gameplay 
from datetime import datetime
now = datetime.now()
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m%y%H%M%S")
print("date and time =", dt_string)


now = 2020-10-28 15:10:02.266588
date and time = 281020151002


In [51]:
datetime.now().strftime("%d")

'28'

In [3]:
import os

files = folders = 0
path = "../capturedata/"
for _, dirnames, filenames in os.walk(path):
  # ^ this idiom means "we won't be using this value"
    files += len(filenames)
    folders += len(dirnames)

print (f"{files} files, {folders} folders")

1082 files, 10 folders


In [21]:
import os
path = "../capturedata/"

# get the directory names
gamefolders = []
for _, dirnames, _ in os.walk(path):
    for name in dirnames:
        gamefolders.append(name)
gamefolders.sort()
gamefolders

['281020153832',
 '281020154935',
 '281020155125',
 '281020155149',
 '281020183330',
 '281020183550',
 '281020183641',
 '281020183702',
 '281020183916',
 '281020184023']

In [60]:
all_actions = []
all_images = []

for folder in gamefolders:
    # get all the files in this directory
    print(folder)
    pathf = path+folder+"/"
    print(pathf)

    # read the variables from img files
    # how many images
    imgcount = len([name for name in os.listdir(pathf) if   os.path.isfile(os.path.join(pathf, name))])
    imgcount = imgcount - 3 #  ignore actions file and last 2 images which often have game over. also note this is indexed from 1.
    
    for i in np.arange(0, imgcount):
        img = cv2.imread(pathf+f'frame_{i}.jpg', cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, None, fx=0.5, fy=0.5)
        all_images.append(img)

    # deal with labels from action.csv
        with open(pathf+'actions.csv', 'r') as actions:
            for line in actions:
                all_actions.append(line.rstrip())
        all_actions = all_actions[0:imgcount]

    



281020153832
../capturedata/281020153832/


In [61]:
len(all_actions)

222

In [62]:
len(all_images)

222